## RAG with discussion on Research Articles

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader

file_path = ".\data\snehPaper.pdf"
loader = UnstructuredPDFLoader(file_path)
docs = loader.load()

c:\Users\sneh.lata\AppData\Local\anaconda3\envs\langgraph-academy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

# get embedding models, create vectorstore, store chunks in vectorstore

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# adding documents in vectorstores
document_ids = vector_store.add_documents(documents=all_splits)

In [11]:
all_splits

[Document(metadata={'source': '.\\data\\snehPaper.pdf', 'start_index': 0}, page_content='5 2 0 2\n\nr a\n\nM 8\n\n]\n\nA F . h t a m\n\n[\n\n3 v 1 3 1 4 0 . 0 1 4 2 : v i X r a\n\nA SEVERAL VARIABLES KOWALSKI-S LODKOWSKI THEOREM FOR TOPOLOGICAL SPACES\n\nJAIKISHAN, SNEH LATA, AND DINESH SINGH'),
 Document(metadata={'source': '.\\data\\snehPaper.pdf', 'start_index': 190}, page_content='Abstract. In this paper, we provide a version of the classical result of Kowalski and S lodkowski that generalizes the famous Gleason-Kahane-˙Zelazko (GKZ) theo- rem by characterizing multiplicative linear functionals amongst all complex-valued functions on a Banach algebra. We ﬁrst characterize maps on A-valued polynomials of several variables that satisfy some conditions, motivated by the result of Kowalski and S lodkowski, as a composition of a multiplicative linear functional on A and a point evaluation on the polynomials, where A is a complex Banach algebra with identity. We then apply it to prove an

In [24]:
#question = "Tell me about Randheer Singh?"
question = "What are the major contribution of this paper?"

In [25]:
# extract chunks which matches with your query
search_results = vector_store.similarity_search_with_score(question, k=10)

# conbine text from differrent chunks into one long string
doc_content = "\n\n".join(doc.page_content for (doc,score) in search_results)

In [26]:
# RAG promt template
prompt_template = """You are an assistant to a Mathematiccian. Use the following pieces of retrieved context to answer the question. Use no more than 500 words to summerize your answer.
Question: {question} 
Context: {context} 
Answer:"""

# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain.chat_models import init_chat_model
llm = init_chat_model("o1-mini", model_provider="openai")

response = llm.invoke(prompt_template.format(
    context=doc_content,
    question=question))
    
print(response.content)

The paper makes several significant contributions to the field of functional analysis, particularly in the study of multiplicative theorems within Hardy spaces and related function spaces. The major contributions can be summarized as follows:

1. **Extension of the Kowalski-Slodkowski (KS) Theorem to Broader Function Spaces**:
   - **Theorem 3.2** is a central advancement that generalizes the KS theorem beyond its traditional scope. It applies to a much wider class of topological spaces, including vector-valued Hardy spaces \( H_p(\mathbb{D}^n, \mathbb{C}^m) \), the Drury-Arveson space, Dirichlet-type spaces over the polydisc \( \mathbb{D}^n \), and the polydisc algebra. This extends the applicability from scalar-valued to vector-valued functions and from single-variable to several-variable contexts.

2. **Elementary Argumentation**:
   - The authors employ elementary arguments to establish their results, contrasting with previous works like [11], which relied on complex function theor

In [27]:
from IPython.display import Markdown
Markdown(response.content)

The paper makes several significant contributions to the field of functional analysis, particularly in the study of multiplicative theorems within Hardy spaces and related function spaces. The major contributions can be summarized as follows:

1. **Extension of the Kowalski-Slodkowski (KS) Theorem to Broader Function Spaces**:
   - **Theorem 3.2** is a central advancement that generalizes the KS theorem beyond its traditional scope. It applies to a much wider class of topological spaces, including vector-valued Hardy spaces \( H_p(\mathbb{D}^n, \mathbb{C}^m) \), the Drury-Arveson space, Dirichlet-type spaces over the polydisc \( \mathbb{D}^n \), and the polydisc algebra. This extends the applicability from scalar-valued to vector-valued functions and from single-variable to several-variable contexts.

2. **Elementary Argumentation**:
   - The authors employ elementary arguments to establish their results, contrasting with previous works like [11], which relied on complex function theory. This simplification not only makes the proofs more accessible but also broadens the potential applicability of the theorems to settings where complex analysis techniques may not be straightforward to apply.

3. **Improvement of Existing Theorems**:
   - **Theorem 2.1** enhances **Theorem 1.7** by showing that its hypotheses are sufficient to extend its conclusions to the entire Hardy space \( H_p(\mathbb{D}) \), similar to the broader conclusions of **Theorem 1.3**. This addresses shortcomings in earlier results by ensuring behavior on unbounded functions within \( H_p(\mathbb{D}) \).

4. **Introduction of Continuity as an Essential Hypothesis**:
   - In **Theorem 3.2**, an additional continuity assumption is introduced, which is demonstrated to be essential through **Example 3.7**. This contrasts with **Theorem 1.6**, where the necessity of continuity was not addressed, thereby refining the conditions under which the KS theorem can be applied effectively.

5. **Generality Beyond Analytic Function Spaces**:
   - Unlike **Theorems 1.6 and 1.7**, **Theorem 3.2** is not restricted to spaces of analytic functions. This broader applicability allows the theorem to be utilized in a wider range of function spaces, enhancing its utility and relevance.

6. **Generalization of Previous Work**:
   - The paper extends earlier work (**Theorem 1.5** from [3]) by incorporating non-vector spaces and vector-valued functions of several variables. This generalization includes topological spaces that are not necessarily vector spaces and extends to vector-valued functions, thereby broadening the theoretical framework established in previous studies.

7. **Simplification Compared to Prior Methods**:
   - The arguments used to prove **Theorem 3.2** are notably simpler than those employed in [12] for **Theorem 1.7**. This methodological improvement not only streamlines the proof process but also makes the results more accessible and easier to apply in various settings.

8. **Comprehensive Comparison with Existing Literature**:
   - Through remarks and comparisons (e.g., Remarks 3.4 and 3.5), the authors position their contributions relative to existing theorems, highlighting advancements and clarifying the extensions made. This contextualization underscores the novelty and importance of their work within the broader landscape of functional analysis research.

In summary, the paper significantly advances the understanding and applicability of multiplicative theorems in functional analysis by:
- Extending the KS theorem to more general and vector-valued function spaces.
- Employing elementary and simplified arguments.
- Introducing essential hypotheses like continuity.
- Improving upon existing theorems to cover comprehensive cases.
- Broadening applicability beyond analytic function spaces.

These contributions collectively enhance the theoretical foundations and practical applications of multiplicative theorems in various function space contexts.